# Tanzania's Water Pumps

The goal for this project is to use logistic regression to create a model which accurately predict's whether a water pump is broken, or working in Tanzania


This is a Predictive Maintnance problem , it allows us to give inspection priority to water pumps our model consideres high-risk.
#### Installing Kaggle

In [2]:
#!pip install kaggle

##### Downloading Datasets using Kaggle API

In [3]:
#!kaggle competitions download -c ds4-predictive-modeling-challenge
#!unzip train_f*.zip
#!unzip test*.zip
#!unzip train_l*.zip
#!rm *.zip
#!chmod +r test_f*.csv
#!chmod +r train*.csv
#!chmod +r train_l*.csv
!ls *.csv

sample_submission.csv train_features.csv
test_features.csv     train_labels.csv


#### Loading CSVs into a Data Frame

In [4]:
import pandas as pd

In [5]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

train_labels = pd.read_csv('train_labels.csv')

#### Explore CSVs

In [6]:
train_features.shape, test_features.shape, train_labels.shape

((59400, 40), (14358, 40), (59400, 2))

In [7]:
col_train_lab = train_labels.columns.values
label = col_train_lab[1]

unique_count = len(train_labels[col_train_lab[1]].unique())
print(f"{unique_count} CLASS CLASSIFICATION PROBLEM")

3 CLASS CLASSIFICATION PROBLEM


In [8]:
col_train = train_features.columns.values
col_test = test_features.columns.values

assert (col_train == col_test).all

columns = col_train

In [9]:
test_features.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe


In [10]:
train_features.describe() #DO PCA ON THESE

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [11]:
train_features.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,...,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,...,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,...,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


#### Cleaning the Data

In [12]:
test_ftr = test_features.set_index('id').sort_index()
train_ftr = train_features.set_index('id').sort_index()
train_lbl = train_labels.set_index('id').sort_index()

# Majority Baseline

In [13]:
count = train_lbl[label].value_counts() #gives counts for each value
perc = train_lbl[label].value_counts(normalize=True)#give % of each value

print(f"Our Majority Baseline is ~{int(perc[0] * 100)}% accurate")

Our Majority Baseline is ~54% accurate


# Extracting Data

# Creating Validation set

A validation set is where you split your train into a train test group, and test the accuracy of your model.

We're going to perform a 3 layer validation set.

## Why care about model validation?

#### Rachel Thomas, [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)

> An all-too-common scenario: a seemingly impressive machine learning model is a complete failure when implemented in production. The fallout includes leaders who are now skeptical of machine learning and reluctant to try it again. How can this happen?

> One of the most likely culprits for this disconnect between results in development vs results in production is a poorly chosen validation set (or even worse, no validation set at all). 

#### Owen Zhang, [Winning Data Science Competitions](https://www.slideshare.net/OwenZhang2/tips-for-data-science-competitions/8)

> Good validation is _more important_ than good models. 

#### James, Witten, Hastie, Tibshirani, [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), Chapter 2.2, Assessing Model Accuracy

> In general, we do not really care how well the method works training on the training data. Rather, _we are interested in the accuracy of the predictions that we obtain when we apply our method to previously unseen test data._ Why is this what we care about? 

> Suppose that we are interested test data in developing an algorithm to predict a stock’s price based on previous stock returns. We can train the method using stock returns from the past 6 months. But we don’t really care how well our method predicts last week’s stock price. We instead care about how well it will predict tomorrow’s price or next month’s price. 

> On a similar note, suppose that we have clinical measurements (e.g. weight, blood pressure, height, age, family history of disease) for a number of patients, as well as information about whether each patient has diabetes. We can use these patients to train a statistical learning method to predict risk of diabetes based on clinical measurements. In practice, we want this method to accurately predict diabetes risk for _future patients_ based on their clinical measurements. We are not very interested in whether or not the method accurately predicts diabetes risk for patients used to train the model, since we already know which of those patients have diabetes.

### TO SPLIT RANDOMLY:

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
train_size = 0.8 #percentage that goes into training set
valid_size = 0.2 #percentage that goes into validation set


x_train = train_features
y_train = train_labels[label]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, train_size=train_size, test_size=valid_size,
    random_state=42, stratify=y_train
)

## random_state is a seed.
## stratify makes the label distribution the same

# Splitting numeric and non-numeric features

In [19]:
x_train_numeric = x_train.select_dtypes('number')
x_val_numeric = x_train.select_dtypes('number')